In [8]:
import os
os.makedirs('Q5_Dataset/train/Cat', exist_ok=True)
os.makedirs('Q5_Dataset/train/Dog', exist_ok=True)
os.makedirs('Q5_Dataset/validation/Cat', exist_ok=True)
os.makedirs('Q5_Dataset/validation/Dog', exist_ok=True)
os.makedirs('Q5_Dataset/test/Cat', exist_ok=True)
os.makedirs('Q5_Dataset/test/Dog', exist_ok=True)

In [4]:
from glob import glob
cats = glob('Q5_Dataset/PetImages/Cat/*.jpg')
dogs = glob('Q5_Dataset/PetImages/Dog/*.jpg')

In [7]:
for data in (cats, dogs):
    num_train = int(len(data) * 0.8)
    num_val = int(len(data) * 0.1)
    # Train Data
    for d in data[:num_train]:
        os.rename(d, 'Q5_Dataset/train/' + d[21:])
    # Validation Data
    for d in data[num_train:num_train + num_val]:
        os.rename(d, 'Q5_Dataset/validation/' + d[21:])
    # Test Data
    for d in data[num_train + num_val:]:
        os.rename(d, 'Q5_Dataset/test/' + d[21:])
        
print('Create 3 folders, 80% for training, 10% for validation, 10% for testing.')

Create 3 folders, 80% for training, 10% for validation, 10% for testing.


In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12515752923335891928
]


In [1]:
import tensorflow as tf

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [7]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6231236470685878402
]


In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [8]:
import torch
import torch.nn as nn
import torchvision
import numpy as np

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.6.0
Torchvision Version:  0.7.0


In [9]:
__all__ = ['ResNet50', 'ResNet101','ResNet152']

In [10]:
def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

In [11]:
class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

In [12]:
class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=1000, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [13]:
def ResNet50():
    return ResNet([3, 4, 6, 3])

In [14]:
def ResNet101():
    return ResNet([3, 4, 23, 3])

In [16]:
if __name__=='__main__':
    #model = torchvision.models.resnet50()
    model = ResNet50()
#     print(model)

    input = torch.randn(1, 3, 224, 224)
    out = model(input)
    print(out.shape)

torch.Size([1, 1000])


In [18]:
input.shape

torch.Size([1, 3, 224, 224])